In [2]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math

from pathlib import Path


## Behavioural Analysis

Soft code:

Load in Screening questionnaires csv file
--> Generate dataframe for each participant with values for questionnaires

Check availability of files for moments A and B for all currently available participants

Read CSV of behavioural file. 
--> Exclude trials with too slow reaction times?
--> Compute total cores and also scores for unique emotions
--> Both valence and arousal (and RT?)

In [3]:
data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')

# Create all relevant directories
date_dir = "24082021"
data_all_dir = os.path.join(data_dir, date_dir)
data_A_dir = os.path.join(data_dir, Path('10001'))
data_B_dir = os.path.join(data_dir, Path('10002'))

screeningFile = "results-survey987313.csv" # Output from LimeSurvey Screening Survey
ppFile = "Participant-Excel.csv" # The excel file that assigns participant numbers etc

data_ppFile_dir = os.path.join(data_all_dir, ppFile)
data_screening_dir = os.path.join(data_all_dir, screeningFile)

df_screening = pd.read_csv(data_screening_dir)
df_ppInfo = pd.read_csv(data_ppFile_dir, encoding = "ISO-8859-1")

# df_screening = df_screening[df_screening['DASS21[DAS21]'].notnull()] # Delete all non-complete data inputs based on completing last question of all questionnaires

In [4]:
df_ppInfo
# df_screening

,ï»¿Participantnummer,email,Entry nummer,Anticonceptie,PMS,datum invullen screening,datum begin laatste menstruatie,datum einde laatste menstruatie,duur menstruatie,duur cyclus,...,Compleet,ParticipantNummer,CSV-A,VIDEO-A,CSV-B,VIDEO-B,DATA GOED,Opmerkingen,Unnamed: 41,Unnamed: 42
0,106.0,laurevandamme32@gmail.com,NaN,NaN,NaN,5-7-2020,10-6-2020,13/06/2020,3.0,28.0,...,NaN,106.0,WAAR,ONWAAR,WAAR,WAAR,ONWAAR,NaN,NaN,NaN
1,107.0,devriendt.sarah@hotmail.com,NaN,NaN,NaN,5-7-2020,21-6-2020,25-6-2020,4.0,28.0,...,NaN,107.0,WAAR,WAAR,NaN,NaN,ONWAAR,NaN,NaN,NaN
2,105.0,ine.opsomer98@gmail.com,NaN,NaN,NaN,5-7-2020,25-6-2020,29-6-2020,4.0,28.0,...,NaN,105.0,WAAR,WAAR,NaN,NaN,ONWAAR,NaN,NaN,NaN
3,108.0,inezvandecasteele@gmail.com,NaN,NaN,NaN,9-7-2020,29-6-2020,5-7-2020,6.0,28.0,...,NaN,108.0,NaN,NaN,NaN,NaN,ONWAAR,NaN,NaN,NaN
4,118.0,Heleen.verplanken@hotmail.com,NaN,NaN,NaN,28-7-2020,6-7-2020,11-7-2020,5.0,28.0,...,NaN,118.0,WAAR,ONWAAR,NaN,NaN,ONWAAR,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Loop over all participant IDs
# Check in subfolder 10001 for files that end with that csv --> save directory
# Check in subfolder 10001 for files that end with that webm --> save directory
# Link dates, give 1 if dates are same, 0 if different
# Check webm file size, give 1 if larger than xxKB

# Check in subfolder 10002 for files that end with that csv --> save directory
# Check in subfolder 10002 for files that end with that webm --> save directory

# df_ppInfo['ID']
os.chdir(data_A_dir)
AVid = glob.glob("*{ParticipantNo}_" + "*.webm", recursive = True) # get all video's in directory
ACsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directry

os.chdir(data_B_dir)
BVid = glob.glob("*{ParticipantNo}_" + "*.webm", recursive = True)
BCsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True)

# os.chdir(data_B_dir)
# BVid = glob.glob("*{ParticipantNo}_*", recursive = True)

In [6]:
print("Sanity check; are we getting files from every folder")
print("Number of AVid: ", len(AVid))
print("Number of BVid: ", len(BVid))
print("Number of ACsv: ", len(ACsv))
print("Number of BCsv: ", len(BCsv))

Sanity check; are we getting files from every folder
Number of AVid:  267
Number of BVid:  295
Number of ACsv:  670
Number of BCsv:  648


In [7]:
# Extract all the ID's from the files to get variables with just the participant numbers
AVidID = []
for index, item in enumerate(AVid):
    AVidID.append(AVid[index].split("video_{ParticipantNo}_")[1][0:3])
AVidID = set(AVidID) # Use set() to get rid of any doubles

ACsvID = []
for index, item in enumerate(ACsv):
    ACsvID.append(ACsv[index].split("data_{ParticipantNo}_")[1][0:3])
ACsvID = set(ACsvID)

BVidID = []
for index, item in enumerate(BVid):
    BVidID.append(BVid[index].split("video_{ParticipantNo}_")[1][0:3])
BVidID = set(BVidID)

BCsvID = []
for index, item in enumerate(BCsv):
    BCsvID.append(BCsv[index].split("data_{ParticipantNo}_")[1][0:3])
BCsvID = set(BCsvID)

In [17]:
# How many files for each moment and type are present, aka: what is the maximum dataset size we can work with here.
print("Number of people with CSV at moment A: ", len(ACsvID))
print("Number of people with both Video and CSV at moment A: ", len(AVidID & ACsvID), "\n")

print("Number of people with CSV at moment B: ", len(BCsvID))
print("Number of people with bothsz Video and CSV at moment B: ", len(BVidID & BCsvID), "\n")

print("Number of people with CSV for BOTH moments: ", len(ACsvID & BCsvID), "\n")
print("Number of people with BOTH Video and CSV for BOTH moments: ", len(AVidID & ACsvID & BVidID & BCsvID))

Number of people with CSV at moment A:  389
Number of people with both Video and CSV at moment A:  263 

Number of people with CSV at moment B:  379
Number of people with bothsz Video and CSV at moment B:  290 

Number of people with CSV for BOTH moments:  332 

Number of people with BOTH Video and CSV for BOTH moments:  185


202

In [18]:
# I don't know what I was doing here last time. So skip for now. 

# columnData.keys()
for ID in df_ppInfo['ID']:
    os.chdir(data_A_dir)
    for file in glob.glob("*{ParticipantNo}_" + str(ID) + "*"):
        count = 0
        print(os.path.join(data_A_dir,file))
        count = count + 1
        if count > 1:
            print("Error: Multiple .webm files for participant " + str(ID))
            break


KeyError: 'ID'

## Video Analysis

Based on available participants and available (behavioural) data for moments A and B
--> Load video's and run OpenFace if export file not yet available

Link video file to behavioural file with matching points (beep) and generate relevant markers for trials
Compute explicit (facial) values per trial and emotion



In [1]:
# Video Starting points / beep noise
startPoints = {"1": 91.6, "2" : 44 , "3" : 14.8}

## Modelling

Compare Moments A and B
Compare explicit vs implicit 

FACTORIAL